In [11]:
import networkx as nx
import os
from utils.global_rules import rules

dataDir = "../data/"
cacheDir = "../data/cache/"

读入图数据，G: 无向图，G_directed: 有向图


In [12]:
import importlib

process = importlib.import_module("process")
importlib.reload(process)

G = process.get_graph()

Loading graph...
Initial Node shape:  (2371558, 4)
Initial Link shape:  (3286986, 3)
After drop nan and duplicates:
Node shape:  (2371558, 4)
Link shape:  (3285490, 3)
Undirected graph loaded.


In [23]:
# Create a Directed Graph from the undirected one
G_directed = nx.DiGraph(G)

运行社区检测


In [13]:
import importlib

community_detection = importlib.import_module("utils.community_detection")
importlib.reload(community_detection)

community_detection.get_community_for_all_groups(G)
# community_detection.get_subgraph_for_all_communities(G)

checking evidence for group 1
community80.json contains Domain_c58c149eec59bb14b0c102a0f303d4c20366926b5c3206555d2937474124beb9
length of community: 30797
community80.json contains Domain_f3554b666038baffa5814c319d3053ee2c2eb30d31d0ef509a1a463386b69845
length of community: 30797
length of community with evidence: 30797

checking evidence for group 2
community155.json contains IP_400c19e584976ff2a35950659d4d148a3d146f1b71692468132b849b0eb8702c
length of community: 842
community155.json contains Domain_b10f98a9b53806ccd3a5ee45676c7c09366545c5b12aa96955cde3953e7ad058
length of community: 842
length of community with evidence: 842

checking evidence for group 3
community443.json contains Whois_Phone_f4a84443fb72da27731660695dd00877e8ce25b264ec418504fface62cdcbbd7
length of community: 3136
community65.json contains Domain_24acfd52f9ceb424d4a2643a832638ce1673b8689fa952d9010dd44949e6b1d9
length of community: 43710
community65.json contains Domain_9c72287c3f9bb38cb0186acf37b7054442b75ac32324df

找出非空 industry 比例最高的两个社区。结果是 74 号社区和 68 号社区，不在已有线索相关的社区中。


In [20]:
# find two communities that have most percentage of non-empty industry nodes
import os
import networkx as nx
import json

import importlib

process = importlib.import_module("process")
importlib.reload(process)

communitySubgraphDir = "../data/community_subgraphs/"


def find_top_communities(top: int = 2):
    def compute_non_empty_industry_percentage(G):
        count = 0
        for node in G.nodes:
            if process.validate_node_industry(G, node):
                count += 1
        return count / len(G.nodes)

    community_files = os.listdir(communitySubgraphDir)
    community_files.sort()
    count_dic = {}
    for file in community_files:
        G = nx.readwrite.json_graph.node_link_graph(
            json.load(open(os.path.join(communitySubgraphDir, file)))
        )
        precentage = compute_non_empty_industry_percentage(G)
        # if # of nodes in the community is less than 500, ignore it
        if len(G.nodes) < 500:
            continue
        count_dic[file] = precentage

    sorted_count_dic = sorted(count_dic.items(), key=lambda kv: kv[1], reverse=True)
    return sorted_count_dic[:top]


find_top_communities()

[('community74.json', 0.7760631834750912),
 ('community68.json', 0.6275346851654215)]

In [24]:
import json
import importlib
import pandas as pd

rules = importlib.import_module("utils.global_rules")
process = importlib.import_module("process")
importlib.reload(rules)
importlib.reload(process)

rules = rules.rules
group_number_assigned = {
    "6": "community74.json",
    "7": "community68.json",
}

# group_to_analyze = ["1", "2", "3", "4", "5"]
group_to_analyze = ["6", "7"]
cacheDir = "../data/cache/"
subgraphDir = "../data/subgraph/"
tempDir = "../data/temp/"
answerDir = "../data/answer/"

params = {
    "1": {
        "pagerankQuantile": 0.95,
        "degreeCentralityQuantile": 0.95,
        "degreeQuantile": 0.8,
        "jump_limit": rules["jump_limit_specified_by_industry"]["1"],
        "countKeepPercent": 0.1,
    },
    "2": {
        "pagerankQuantile": 0.92,
        "degreeCentralityQuantile": 0.92,
        "degreeQuantile": 0.9,
        "jump_limit": rules["jump_limit_specified_by_industry"]["2"],
        "countKeepPercent": 0.6,
    },
    "3": {
        "pagerankQuantile": 0.95,
        "degreeCentralityQuantile": 0.95,
        "degreeQuantile": 0.94,
        "jump_limit": rules["jump_limit_specified_by_industry"]["3"],
        "countKeepPercent": 0.4,
    },
    "4": {
        "pagerankQuantile": 0.9,
        "degreeCentralityQuantile": 0.88,
        "degreeQuantile": 0.82,
        "jump_limit": rules["jump_limit_specified_by_industry"]["4"],
        "countKeepPercent": 0.4,
    },
    "5": {
        "pagerankQuantile": 0.7,
        "degreeCentralityQuantile": 0.7,
        "degreeQuantile": 0.85,
        "jump_limit": rules["jump_limit_specified_by_industry"]["5"],
        "countKeepPercent": 0.4,
    },
    "6": {
        "pagerankQuantile": 0.98,
        "degreeCentralityQuantile": 0.98,
        "degreeQuantile": 0.98,
        "jump_limit": rules["jump_limit_specified_by_industry"]["5"],
        "countKeepPercent": 0.4,
    },
    "7": {
        "pagerankQuantile": 0.98,
        "degreeCentralityQuantile": 0.98,
        "degreeQuantile": 0.98,
        "jump_limit": rules["jump_limit_specified_by_industry"]["5"],
        "countKeepPercent": 0.4,
    },
}


def export_subgraph(graph, group_str: str):
    # transform the graph to directed graph
    # graph = graph.to_directed()
    # export the graph to json
    data = nx.readwrite.json_graph.node_link_data(graph)
    # Keep only the nodes and edges
    data = {"nodes": data["nodes"], "edges": data["links"]}
    json.dump(
        data,
        open(tempDir + "group_" + group_str + ".json", "w"),
        indent=4,
    )

    json.dump(
        nx.readwrite.json_graph.node_link_data(graph),
        open(subgraphDir + "group_" + group_str + ".json", "w"),
        indent=4,
    )

    # Create a directory in subgraphDir
    if not os.path.exists(subgraphDir + group_str):
        os.makedirs(subgraphDir + group_str)

    node_data = pd.DataFrame(data["nodes"])
    node_data.to_csv(subgraphDir + group_str + "/nodes.csv", index=False)
    link_data = pd.DataFrame(data["edges"])
    link_data.to_csv(subgraphDir + group_str + "/links.csv", index=False)

    if not os.path.exists(answerDir + group_str):
        os.makedirs(answerDir + group_str)

    # export node in format: id, name, type, industry
    node_data[["id", "name", "type", "industry"]].to_csv(
        answerDir + group_str + "/nodes.csv", index=False
    )
    # export link in format: relation, source, target
    link_data[["relation", "source", "target"]].to_csv(
        answerDir + group_str + "/links.csv", index=False
    )

    return


def analyse_group(group_str: str):
    print("analyse group: ", group_str)
    if group_str not in group_number_assigned:
        # get the evidence
        evidence = rules["evidence"][group_str]
        subgraph = process.get_subgraph(G, evidence, params[group_str]["jump_limit"])
    else:
        subgraph = nx.readwrite.json_graph.node_link_graph(
            json.load(
                open(
                    os.path.join(communitySubgraphDir, group_number_assigned[group_str])
                )
            )
        )
    print("#nodes in subgraph: ", subgraph.number_of_nodes())
    print("#edges in subgraph: ", subgraph.number_of_edges())

    process.set_core(subgraph)

    subgraph_filtered = process.filter_subgraph(
        subgraph,
        pagerankQuantile=params[group_str]["pagerankQuantile"],
        degreeCentralityQuantile=params[group_str]["degreeCentralityQuantile"],
        degreeQuantile=params[group_str]["degreeQuantile"],
        countKeepPercent=params[group_str]["countKeepPercent"],
        # scaleThreshold=rules["net_limit"][rules["scale"][group_str]]["node"],
        # verbose=True,
    )
    export_subgraph(subgraph_filtered, group_str)

    print()
    return


for group in group_to_analyze:
    analyse_group(group)

# analyse_group("2")

analyse group:  6
#nodes in subgraph:  8230
#edges in subgraph:  14258
Nodes left:  764
Edges left:  1229
Nodes with non-empty industry: 748 / 750

analyse group:  7
#nodes in subgraph:  6559
#edges in subgraph:  8583
Nodes left:  993
Edges left:  1088
Nodes with non-empty industry: 962 / 967



In [25]:
import importlib

group_to_analyze = ["1", "2", "3", "4", "5", "6", "7"]
count = importlib.import_module("utils.count")
keypath = importlib.import_module("utils.key_nodes_path")
importlib.reload(count)
importlib.reload(keypath)

count.count_type_for_all_groups(group_to_analyze)
keypath.find_key_path_for_all_groups(group_to_analyze)

group:  6
group:  7
find key path for group:  6
#nodes:  764
#edges:  1229
find key path for group:  7
#nodes:  993
#edges:  1088
